In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('/content/drive/MyDrive/JU PROJ/Reddit/GENERALRedditTesla.csv')
df

,upvotes,Date,Post,sub
0,1,2022-05-25,**Intro**\n\nGreetings you filthy animals! In ...,stocks
1,1,2022-05-25,Disclosure: I am long Tesla stock.\n\nI think ...,stocks
2,1,2022-05-24,"The three major indexes fell collectively, the...",stocks
3,1,2022-05-24,"The three major indexes fell collectively, the...",stocks
4,1,2022-05-24,The three major U.S. stock indexes rose collec...,stocks
...,...,...,...,...
11577,1,2021-03-11,People. \n\nThis post is going to assume you a...,cryptocurrency
11578,1,2021-03-11,I think we should ban posts that add nothing a...,cryptocurrency
11579,1,2021-03-10,I seen many posts here talking about bitcoin l...,cryptocurrency
11580,1,2021-03-10,I have this theory about Elon and Crypto that ...,cryptocurrency


In [6]:
df = df.drop(['upvotes', 'sub'], axis = 1)

In [7]:
df['Date'] = pd.to_datetime(df['Date']).dt.date
df

,Date,Post
0,2022-05-25,**Intro**\n\nGreetings you filthy animals! In ...
1,2022-05-25,Disclosure: I am long Tesla stock.\n\nI think ...
2,2022-05-24,"The three major indexes fell collectively, the..."
3,2022-05-24,"The three major indexes fell collectively, the..."
4,2022-05-24,The three major U.S. stock indexes rose collec...
...,...,...
11577,2021-03-11,People. \n\nThis post is going to assume you a...
11578,2021-03-11,I think we should ban posts that add nothing a...
11579,2021-03-10,I seen many posts here talking about bitcoin l...
11580,2021-03-10,I have this theory about Elon and Crypto that ...


In [8]:
df.columns = ['Date', 'Tweet']
df

,Date,Tweet
0,2022-05-25,**Intro**\n\nGreetings you filthy animals! In ...
1,2022-05-25,Disclosure: I am long Tesla stock.\n\nI think ...
2,2022-05-24,"The three major indexes fell collectively, the..."
3,2022-05-24,"The three major indexes fell collectively, the..."
4,2022-05-24,The three major U.S. stock indexes rose collec...
...,...,...
11577,2021-03-11,People. \n\nThis post is going to assume you a...
11578,2021-03-11,I think we should ban posts that add nothing a...
11579,2021-03-10,I seen many posts here talking about bitcoin l...
11580,2021-03-10,I have this theory about Elon and Crypto that ...


In [9]:
def tweets_per_day(df):
    df['Date'] = pd.to_datetime(df['Date'])
#   return df[['Tweets']].groupby(df['Date'].dt.date).count()
    tweet = df['Tweet'].groupby(df['Date'].dt.date).count() 
    d1 = pd.DataFrame(tweet)
    return d1
  # if you want output to be `Series`

df1 = tweets_per_day(df)
df1 = df1.reset_index(level=0)
df1

,Date,Tweet
0,2011-05-11,1
1,2011-07-11,1
2,2011-11-12,1
3,2012-01-14,1
4,2012-01-17,1
...,...,...
2045,2022-05-21,4
2046,2022-05-22,3
2047,2022-05-23,2
2048,2022-05-24,9


In [10]:
#df = df.groupby(['Date'])['Tweet'].apply(lambda x: '. '.join(x.astype(str))).reset_index()
df

,Date,Tweet
0,2022-05-25,**Intro**\n\nGreetings you filthy animals! In ...
1,2022-05-25,Disclosure: I am long Tesla stock.\n\nI think ...
2,2022-05-24,"The three major indexes fell collectively, the..."
3,2022-05-24,"The three major indexes fell collectively, the..."
4,2022-05-24,The three major U.S. stock indexes rose collec...
...,...,...
11577,2021-03-11,People. \n\nThis post is going to assume you a...
11578,2021-03-11,I think we should ban posts that add nothing a...
11579,2021-03-10,I seen many posts here talking about bitcoin l...
11580,2021-03-10,I have this theory about Elon and Crypto that ...


In [11]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax

In [12]:
import nltk 
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re
import math

stop_words = stopwords.words('english')
stop_words.remove("not")
stemmer = SnowballStemmer('english')

text_cleaning_re = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"

def preprocess(text, stem=False):
  text = re.sub(text_cleaning_re, ' ', str(text).lower()).strip()
  tokens = []
  for token in text.split():
    if token not in stop_words:
      if stem:
        tokens.append(stemmer.stem(token))
      else:
        tokens.append(token)
  return " ".join(tokens)

df.Tweets = df.Tweet.apply(lambda x: preprocess(x))
df

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access


,Date,Tweet
0,2022-05-25,**Intro**\n\nGreetings you filthy animals! In ...
1,2022-05-25,Disclosure: I am long Tesla stock.\n\nI think ...
2,2022-05-24,"The three major indexes fell collectively, the..."
3,2022-05-24,"The three major indexes fell collectively, the..."
4,2022-05-24,The three major U.S. stock indexes rose collec...
...,...,...
11577,2021-03-11,People. \n\nThis post is going to assume you a...
11578,2021-03-11,I think we should ban posts that add nothing a...
11579,2021-03-10,I seen many posts here talking about bitcoin l...
11580,2021-03-10,I have this theory about Elon and Crypto that ...


In [13]:
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [14]:
neg = []
neu = []
pos = []
i = 0
def normalize(score, alpha=15):
    """
    Normalize the score to be between -1 and 1 using an alpha that
    approximates the max expected value
    """
    norm_score = score/math.sqrt((score*score) + alpha)
    return norm_score

for tweet in df.Tweet:
  i = i+1
  encoded_tweet = tokenizer(tweet,padding=True, truncation=True,max_length=250, add_special_tokens = True, return_tensors='pt')
  output = model(**encoded_tweet)
  scores = output[0][0].detach().numpy()
  normScores = softmax(scores)  
  neg.append(normScores[0])
  neu.append(normScores[1])
  pos.append(normScores[2])
  print(i,"--> ",normScores)

Streaming output truncated to the last 5000 lines.
6583 -->  [0.08466998 0.04856862 0.86676145]
6584 -->  [0.05883307 0.01773536 0.92343163]
6585 -->  [0.03347271 0.05022985 0.91629744]
6586 -->  [0.04998954 0.02381047 0.9261999 ]
6587 -->  [0.0457989  0.14551884 0.80868226]
6588 -->  [0.08743367 0.01439836 0.8981681 ]
6589 -->  [0.07665543 0.01757833 0.90576625]
6590 -->  [0.03058694 0.49613896 0.47327405]
6591 -->  [0.04351543 0.05688964 0.899595  ]
6592 -->  [0.07282565 0.18288878 0.74428564]
6593 -->  [0.03718985 0.55375385 0.40905625]
6594 -->  [0.8798721  0.06223778 0.05789005]
6595 -->  [0.02787763 0.05846346 0.91365886]
6596 -->  [0.8796285  0.00870662 0.11166488]
6597 -->  [0.07057625 0.01500047 0.9144232 ]
6598 -->  [0.06314752 0.2840818  0.65277076]
6599 -->  [0.8641743  0.00789432 0.12793134]
6600 -->  [0.05690993 0.03474428 0.9083459 ]
6601 -->  [0.08395547 0.01341785 0.9026266 ]
6602 -->  [0.05717405 0.02260154 0.92022437]
6603 -->  [0.05260724 0.31330892 0.6340838 ]
6604

In [15]:
df['negative'] = neg
df['neutral'] = neu
df['positive'] = pos
df

,Date,Tweet,negative,neutral,positive
0,2022-05-25,**Intro**\n\nGreetings you filthy animals! In ...,0.038985,0.057603,0.903412
1,2022-05-25,Disclosure: I am long Tesla stock.\n\nI think ...,0.035307,0.315280,0.649413
2,2022-05-24,"The three major indexes fell collectively, the...",0.010727,0.967651,0.021622
3,2022-05-24,"The three major indexes fell collectively, the...",0.010455,0.969561,0.019984
4,2022-05-24,The three major U.S. stock indexes rose collec...,0.922591,0.017632,0.059777
...,...,...,...,...,...
11577,2021-03-11,People. \n\nThis post is going to assume you a...,0.046429,0.053878,0.899693
11578,2021-03-11,I think we should ban posts that add nothing a...,0.035542,0.068477,0.895981
11579,2021-03-10,I seen many posts here talking about bitcoin l...,0.090351,0.195663,0.713985
11580,2021-03-10,I have this theory about Elon and Crypto that ...,0.058109,0.026731,0.915159


In [16]:
dfx = pd.DataFrame()
po = []
nu = []
ne = []
for i in range (0,2050):

  d = df['Date'][i]

  x = df[df['Date'] == d]
  po.append(x['positive'].mean())
  nu.append(x['neutral'].mean())
  ne.append(x['negative'].mean())
dfx['Date'] = df1['Date']
dfx['positive'] = po
dfx['negative'] = ne
dfx['neutral'] = nu
dfx

,Date,positive,negative,neutral
0,2011-05-11,0.544262,0.028986,0.426751
1,2011-07-11,0.544262,0.028986,0.426751
2,2011-11-12,0.286538,0.181743,0.531720
3,2012-01-14,0.286538,0.181743,0.531720
4,2012-01-17,0.286538,0.181743,0.531720
...,...,...,...,...
2045,2022-05-21,0.732631,0.170854,0.096515
2046,2022-05-22,0.732631,0.170854,0.096515
2047,2022-05-23,0.732631,0.170854,0.096515
2048,2022-05-24,0.732631,0.170854,0.096515


In [17]:
# Set date as Index
dfx = dfx.set_index('Date')
dfx = dfx.sort_index()
dfx

,positive,negative,neutral
Date,,,
2011-05-11,0.544262,0.028986,0.426751
2011-07-11,0.544262,0.028986,0.426751
2011-11-12,0.286538,0.181743,0.531720
2012-01-14,0.286538,0.181743,0.531720
2012-01-17,0.286538,0.181743,0.531720
...,...,...,...
2022-05-21,0.732631,0.170854,0.096515
2022-05-22,0.732631,0.170854,0.096515
2022-05-23,0.732631,0.170854,0.096515


In [18]:
from pandas_datareader import data as pdr
import yfinance as yf

yf.pdr_override()

stock_price = pdr.get_data_yahoo("TSLA", start = '2017-01-01', end = '2022-05-07')
print(len(stock_price))
stock_price

[*********************100%***********************]  1 of 1 completed
1346


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-01-03,42.972000,44.066002,42.192001,43.397999,43.397999,29616500
2017-01-04,42.950001,45.599998,42.862000,45.397999,45.397999,56067500
2017-01-05,45.284000,45.495998,44.389999,45.349998,45.349998,29558500
2017-01-06,45.386002,46.062000,45.090000,45.801998,45.801998,27639500
2017-01-09,45.793999,46.383999,45.599998,46.256001,46.256001,19897500
...,...,...,...,...,...,...
2022-05-02,860.770020,906.359985,848.030029,902.940002,902.940002,25260500
2022-05-03,903.179993,924.080017,888.590027,909.250000,909.250000,21236500
2022-05-04,903.940002,955.500000,885.280029,952.619995,952.619995,27214600


In [19]:
stock_price = stock_price.drop_duplicates()

# dropping the zero values from volume
stock_price = stock_price[stock_price['Volume'] != 0]

# filtering the important columns required
stock_price = stock_price.filter(['Date', 'Close', 'Open', 'High', 'Low', 'Volume'])

# sorting the data according to the index i.e 'Date'
stock_price = stock_price.sort_index(ascending=True, axis=0)
stock_price

,Close,Open,High,Low,Volume
Date,,,,,
2017-01-03,43.397999,42.972000,44.066002,42.192001,29616500
2017-01-04,45.397999,42.950001,45.599998,42.862000,56067500
2017-01-05,45.349998,45.284000,45.495998,44.389999,29558500
2017-01-06,45.801998,45.386002,46.062000,45.090000,27639500
2017-01-09,46.256001,45.793999,46.383999,45.599998,19897500
...,...,...,...,...,...
2022-05-02,902.940002,860.770020,906.359985,848.030029,25260500
2022-05-03,909.250000,903.179993,924.080017,888.590027,21236500
2022-05-04,952.619995,903.940002,955.500000,885.280029,27214600


In [20]:
stock_data = pd.merge(stock_price,dfx, how='left', left_index=True, right_index=True)

stock_data

,Close,Open,High,Low,Volume,positive,negative,neutral
Date,,,,,,,,
2017-01-03,43.397999,42.972000,44.066002,42.192001,29616500,0.656125,0.175266,0.168609
2017-01-04,45.397999,42.950001,45.599998,42.862000,56067500,0.573474,0.386913,0.039613
2017-01-05,45.349998,45.284000,45.495998,44.389999,29558500,0.271435,0.434826,0.293739
2017-01-06,45.801998,45.386002,46.062000,45.090000,27639500,NaN,NaN,NaN
2017-01-09,46.256001,45.793999,46.383999,45.599998,19897500,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2022-05-02,902.940002,860.770020,906.359985,848.030029,25260500,0.732631,0.170854,0.096515
2022-05-03,909.250000,903.179993,924.080017,888.590027,21236500,0.732631,0.170854,0.096515
2022-05-04,952.619995,903.940002,955.500000,885.280029,27214600,NaN,NaN,NaN


In [21]:
stock_data.sort_index(axis = 1)
stock_data

,Close,Open,High,Low,Volume,positive,negative,neutral
Date,,,,,,,,
2017-01-03,43.397999,42.972000,44.066002,42.192001,29616500,0.656125,0.175266,0.168609
2017-01-04,45.397999,42.950001,45.599998,42.862000,56067500,0.573474,0.386913,0.039613
2017-01-05,45.349998,45.284000,45.495998,44.389999,29558500,0.271435,0.434826,0.293739
2017-01-06,45.801998,45.386002,46.062000,45.090000,27639500,NaN,NaN,NaN
2017-01-09,46.256001,45.793999,46.383999,45.599998,19897500,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2022-05-02,902.940002,860.770020,906.359985,848.030029,25260500,0.732631,0.170854,0.096515
2022-05-03,909.250000,903.179993,924.080017,888.590027,21236500,0.732631,0.170854,0.096515
2022-05-04,952.619995,903.940002,955.500000,885.280029,27214600,NaN,NaN,NaN


In [22]:
stock_data.isna().sum()

Close         0
Open          0
High          0
Low           0
Volume        0
positive    224
negative    224
neutral     224
dtype: int64

In [23]:
#stock_data.drop(['Tweet'], inplace=True, axis=1)

# rearranging the columns of the whole stock_data
stock_data = stock_data[['Close', 'negative', 'neutral', 'positive', 'Open', 'High', 'Low', 'Volume']]

# displaying the final stock_data
stock_data

,Close,negative,neutral,positive,Open,High,Low,Volume
Date,,,,,,,,
2017-01-03,43.397999,0.175266,0.168609,0.656125,42.972000,44.066002,42.192001,29616500
2017-01-04,45.397999,0.386913,0.039613,0.573474,42.950001,45.599998,42.862000,56067500
2017-01-05,45.349998,0.434826,0.293739,0.271435,45.284000,45.495998,44.389999,29558500
2017-01-06,45.801998,NaN,NaN,NaN,45.386002,46.062000,45.090000,27639500
2017-01-09,46.256001,NaN,NaN,NaN,45.793999,46.383999,45.599998,19897500
...,...,...,...,...,...,...,...,...
2022-05-02,902.940002,0.170854,0.096515,0.732631,860.770020,906.359985,848.030029,25260500
2022-05-03,909.250000,0.170854,0.096515,0.732631,903.179993,924.080017,888.590027,21236500
2022-05-04,952.619995,NaN,NaN,NaN,903.940002,955.500000,885.280029,27214600


In [24]:
stock_data.to_csv('/content/drive/MyDrive/JU PROJ/Reddit/GENERAL_FINBERT_Reddit_data.csv')